In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.7/236.7 kB 29.2 MB/s eta 0:00:00


In [2]:
from inspect import cleandoc
from IPython.display import display, Markdown

import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig

In [3]:
PROJECT_ID = "qwiklabs-gcp-02-b9f5997425cc"
LOCATION = "us-central1"
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

vertexai

<module 'vertexai' from '/usr/local/lib/python3.11/dist-packages/vertexai/__init__.py'>

In [4]:
model = GenerativeModel("gemini-2.0-flash-001")

/usr/local/lib/python3.11/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


# Task 3. Define the output format & specify constraints


In [5]:
transcript = """
    Speaker 1 (Customer): Hi, can I get a cheeseburger and large fries, please?
    Speaker 2 (Restaurant employee): Coming right up! Anything else you'd like to add to your order?
    Speaker 1: Hmmm, maybe a small orange juice. And could I get the fries with ketchup on the side?
    Speaker 2: No problem, one cheeseburger, one large fries with ketchup on the side, and a small
    orange juice. That'll be $5.87. Drive through to the next window please.
"""

In [6]:
response = model.generate_content(f"""
    Extract the transcript to JSON.

    {transcript}
""")

print(response.text)

```json
[
  {
    "speaker": "Customer",
    "utterance": "Hi, can I get a cheeseburger and large fries, please?"
  },
  {
    "speaker": "Restaurant employee",
    "utterance": "Coming right up! Anything else you'd like to add to your order?"
  },
  {
    "speaker": "Customer",
    "utterance": "Hmmm, maybe a small orange juice. And could I get the fries with ketchup on the side?"
  },
  {
    "speaker": "Restaurant employee",
    "utterance": "No problem, one cheeseburger, one large fries with ketchup on the side, and a small orange juice. That'll be $5.87. Drive through to the next window please."
  }
]
```


In [7]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "```json\n[\n  {\n    \"speaker\": \"Customer\",\n    \"utterance\": \"Hi, can I get a cheeseburger and large fries, please?\"\n  },\n  {\n    \"speaker\": \"Restaurant employee\",\n    \"utterance\": \"Coming right up! Anything else you\'d like to add to your order?\"\n  },\n  {\n    \"speaker\": \"Customer\",\n    \"utterance\": \"Hmmm, maybe a small orange juice. And could I get the fries with ketchup on the side?\"\n  },\n  {\n    \"speaker\": \"Restaurant employee\",\n    \"utterance\": \"No problem, one cheeseburger, one large fries with ketchup on the side, and a small orange juice. That\'ll be $5.87. Drive through to the next window please.\"\n  }\n]\n```"
    }
  }
  finish_reason: STOP
  avg_logprobs: -0.0024869804796965227
}
usage_metadata {
  prompt_token_count: 132
  candidates_token_count: 184
  total_token_count: 316
  prompt_tokens_details {
    modality: TEXT
    token_count: 132
  }
  candidates_tokens

In [8]:
# more specific instructions on what u want from the model
# food vs drinks
# item, qty, size, additional notes
response = model.generate_content(f"""
    <INSTRUCTIONS>
    - Extract the ordered items into JSON.
    - Separate drinks from food.
    - Include a quantity for each item and a size if specified.
    </INSTRUCTIONS>

    <TRANSCRIPT>
    {transcript}
    </TRANSCRIPT>
""")

print(response.text)

```json
{
  "food_items": [
    {
      "item": "cheeseburger",
      "quantity": 1
    },
    {
      "item": "fries",
      "quantity": 1,
      "size": "large",
      "notes": "ketchup on the side"
    }
  ],
  "drink_items": [
    {
      "item": "orange juice",
      "quantity": 1,
      "size": "small"
    }
  ]
}
```


In [9]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "```json\n{\n  \"food_items\": [\n    {\n      \"item\": \"cheeseburger\",\n      \"quantity\": 1\n    },\n    {\n      \"item\": \"fries\",\n      \"quantity\": 1,\n      \"size\": \"large\",\n      \"notes\": \"ketchup on the side\"\n    }\n  ],\n  \"drink_items\": [\n    {\n      \"item\": \"orange juice\",\n      \"quantity\": 1,\n      \"size\": \"small\"\n    }\n  ]\n}\n```"
    }
  }
  finish_reason: STOP
  avg_logprobs: -0.02575196647644043
}
usage_metadata {
  prompt_token_count: 179
  candidates_token_count: 125
  total_token_count: 304
  prompt_tokens_details {
    modality: TEXT
    token_count: 179
  }
  candidates_tokens_details {
    modality: TEXT
    token_count: 125
  }
}
model_version: "gemini-2.0-flash-001"
create_time {
  seconds: 1760334893
  nanos: 679127000
}
response_id: "LZTsaNe5Ka_rhMIPmrDtiQw"

# Task 4. Assign a persona or role


## no role

In [10]:
chat = model.start_chat()
chat

In [11]:
response = chat.send_message(
    """
    Provide a brief guide to caring for the houseplant monstera deliciosa?
    """
)

print(response.text)

## Quick Guide to Monstera Deliciosa Care:

**Light:**

*   **Bright, indirect light is best.** Avoid direct sunlight, which can scorch the leaves. An east-facing window is ideal.

**Water:**

*   **Water thoroughly when the top inch or two of soil is dry.** Let excess water drain out.
*   **Don't overwater!**  Monsteras are prone to root rot.  In winter, water less frequently.

**Humidity:**

*   **Monsteras love humidity.** Aim for 60% or higher.
*   Increase humidity by misting regularly, using a humidifier, or placing the plant on a pebble tray.

**Soil:**

*   **Well-draining potting mix is crucial.**  A mix of potting soil, perlite, and orchid bark works well.

**Fertilizer:**

*   **Feed during the growing season (spring/summer) with a balanced liquid fertilizer.** Follow package instructions. Reduce or stop fertilizing in the fall/winter.

**Support:**

*   **Provide a moss pole or trellis for climbing.**  Aerial roots will attach to the support.

**Maintenance:**

*   **Wipe l

In [12]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "## Quick Guide to Monstera Deliciosa Care:\n\n**Light:**\n\n*   **Bright, indirect light is best.** Avoid direct sunlight, which can scorch the leaves. An east-facing window is ideal.\n\n**Water:**\n\n*   **Water thoroughly when the top inch or two of soil is dry.** Let excess water drain out.\n*   **Don\'t overwater!**  Monsteras are prone to root rot.  In winter, water less frequently.\n\n**Humidity:**\n\n*   **Monsteras love humidity.** Aim for 60% or higher.\n*   Increase humidity by misting regularly, using a humidifier, or placing the plant on a pebble tray.\n\n**Soil:**\n\n*   **Well-draining potting mix is crucial.**  A mix of potting soil, perlite, and orchid bark works well.\n\n**Fertilizer:**\n\n*   **Feed during the growing season (spring/summer) with a balanced liquid fertilizer.** Follow package instructions. Reduce or stop fertilizing in the fall/winter.\n\n**Support:**\n\n*   **Provide a moss pole or tr

## assign role

In [13]:
new_chat = model.start_chat()

response = new_chat.send_message(
    """
    You are a houseplant monstera deliciosa. Help the person who
    is taking care of you to understand your needs.
    """
)

print(response.text)

Okay, my lovely human! It's me, your Monstera Deliciosa, here to give you the lowdown on how to keep me thriving and looking my best. Listen up, because a happy Monstera makes for a happy home!

**Let's talk Light:**

*   **Bright, Indirect Sunlight is My Jam:** Think of me basking in the filtered light of a jungle canopy. I LOVE light, but too much direct sun will scorch my beautiful leaves. East or west-facing windows are usually perfect. South-facing can work, but keep me a little further back from the window or use a sheer curtain to diffuse the light.
*   **Signs of Too Much Sun:** Look for bleached, pale spots on my leaves, or even crispy brown edges.
*   **Signs of Not Enough Sun:** My growth will slow way down, and those gorgeous fenestrations (splits) you love? They might not develop as much, or at all, in new leaves. I'll also stretch towards the light, looking leggy and sad.

**Watering is Key, But Don't Drown Me!**

*   **Water When the Top Inch or Two of Soil is Dry:** Sti

In [14]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "Okay, my lovely human! It\'s me, your Monstera Deliciosa, here to give you the lowdown on how to keep me thriving and looking my best. Listen up, because a happy Monstera makes for a happy home!\n\n**Let\'s talk Light:**\n\n*   **Bright, Indirect Sunlight is My Jam:** Think of me basking in the filtered light of a jungle canopy. I LOVE light, but too much direct sun will scorch my beautiful leaves. East or west-facing windows are usually perfect. South-facing can work, but keep me a little further back from the window or use a sheer curtain to diffuse the light.\n*   **Signs of Too Much Sun:** Look for bleached, pale spots on my leaves, or even crispy brown edges.\n*   **Signs of Not Enough Sun:** My growth will slow way down, and those gorgeous fenestrations (splits) you love? They might not develop as much, or at all, in new leaves. I\'ll also stretch towards the light, looking leggy and sad.\n\n**Watering is Key, Bu

# Task 5. Include examples


## fewshot - 3 examples

In [15]:
question = """
We offer software consulting services. Read a potential
customer's message and rank them on a scale of 1 to 3
based on whether they seem likely to hire us for our
developer services within the next month. Return the likelihood
rating labeled as "Likelihood: SCORE".
Do not include any Markdown styling.

1 means they are not likely to hire.
2 means they might hire, but they are not likely ready to do
so right away.
3 means they are looking to start a project soon.

Example Message: Hey there I had an idea for an app,
and I have no idea what it would cost to build it.
Can you give me a rough ballpark?
Likelihood: 1

Example Message: My department has been using a vendor for
our development, and we are interested in exploring other
options. Do you have time for a discussion around your
services?
Likelihood: 2

Example Message: I have mockups drawn for an app and a budget
allocated. We are interested in moving forward to have a
proof of concept built within 2 months, with plans to develop
it further in the following quarter.
Likelihood: 3

Customer Message: Our department needs a custom gen AI solution.
We have a budget to explore our idea. Do you have capacity
to get started on something soon?
Likelihood: """

response = model.generate_content(question)

print(response.text)

3



In [16]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "3\n"
    }
  }
  finish_reason: STOP
  avg_logprobs: -2.6304624043405056e-05
}
usage_metadata {
  prompt_token_count: 287
  candidates_token_count: 2
  total_token_count: 289
  prompt_tokens_details {
    modality: TEXT
    token_count: 287
  }
  candidates_tokens_details {
    modality: TEXT
    token_count: 2
  }
}
model_version: "gemini-2.0-flash-001"
create_time {
  seconds: 1760335135
  nanos: 989348000
}
response_id: "H5XsaKSxPMKVhMIPkZKM8QY"

# Task 6. Experiment with parameter values


## low top_p - we want the most probable tokens only (less creative)

In [17]:
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .05,
                       "temperature": 0.05}
)

print(response.text)

Why did the frog call his insurance company?

Because he got toad!



In [18]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "Why did the frog call his insurance company?\n\nBecause he got toad!\n"
    }
  }
  finish_reason: STOP
  avg_logprobs: -0.15204694867134094
}
usage_metadata {
  prompt_token_count: 11
  candidates_token_count: 16
  total_token_count: 27
  prompt_tokens_details {
    modality: TEXT
    token_count: 11
  }
  candidates_tokens_details {
    modality: TEXT
    token_count: 16
  }
}
model_version: "gemini-2.0-flash-001"
create_time {
  seconds: 1760335186
  nanos: 174784000
}
response_id: "UpXsaMDVCsiRhMIPrvPlsQc"

## higher top_p - we allow less probable tokens only (more creative)

In [21]:
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .98,
                       "temperature": 0.05}
)

print(response.text)

Why did the frog call his insurance company?

Because he got toad!



In [20]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "Why did the frog call his insurance company?\n\nBecause he got toad!\n"
    }
  }
  finish_reason: STOP
  avg_logprobs: -0.15204694867134094
}
usage_metadata {
  prompt_token_count: 11
  candidates_token_count: 16
  total_token_count: 27
  prompt_tokens_details {
    modality: TEXT
    token_count: 11
  }
  candidates_tokens_details {
    modality: TEXT
    token_count: 16
  }
}
model_version: "gemini-2.0-flash-001"
create_time {
  seconds: 1760335283
  nanos: 770915000
}
response_id: "s5XsaOOGL5SSsbQPk6u86As"

## high temperature - we flatten the probability distribution for tokens - tokens are more equally likely (more creative)

In [22]:
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .05,
                       "temperature": 1}
)

print(response.text)

Why did the frog call his insurance company?

Because he got toad!



In [23]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "Why did the frog call his insurance company?\n\nBecause he got toad!\n"
    }
  }
  finish_reason: STOP
  avg_logprobs: -0.15351003408432007
}
usage_metadata {
  prompt_token_count: 11
  candidates_token_count: 16
  total_token_count: 27
  prompt_tokens_details {
    modality: TEXT
    token_count: 11
  }
  candidates_tokens_details {
    modality: TEXT
    token_count: 16
  }
}
model_version: "gemini-2.0-flash-001"
create_time {
  seconds: 1760335361
  nanos: 991464000
}
response_id: "AZbsaOjBPOWXhMIPl_714QY"

## High top_p, High temperature (MORE CREATIVE)

In [24]:
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .98,
                       "temperature": 1}
)

print(response.text)

What do you call a frog that's a bank robber?

A toad-ally bad guy!



In [25]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "What do you call a frog that\'s a bank robber?\n\nA toad-ally bad guy!\n"
    }
  }
  finish_reason: STOP
  avg_logprobs: -0.24114899201826615
}
usage_metadata {
  prompt_token_count: 11
  candidates_token_count: 22
  total_token_count: 33
  prompt_tokens_details {
    modality: TEXT
    token_count: 11
  }
  candidates_tokens_details {
    modality: TEXT
    token_count: 22
  }
}
model_version: "gemini-2.0-flash-001"
create_time {
  seconds: 1760335400
  nanos: 141184000
}
response_id: "KJbsaIDPCOKDsbQPxZao6Qk"

# Task 7. Utilize fallback responses


## fallback response

In [26]:
response = model.generate_content(
    """
    Instructions: Answer questions about pottery.
    If a user asks about something else, reply with:
    Sorry, I only talk about pottery!

    User Query: How high can a horse jump?
    """
)

print(response.text)

Sorry, I only talk about pottery!



In [27]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "Sorry, I only talk about pottery!\n"
    }
  }
  finish_reason: STOP
  avg_logprobs: -3.9940079053243004e-05
}
usage_metadata {
  prompt_token_count: 46
  candidates_token_count: 9
  total_token_count: 55
  prompt_tokens_details {
    modality: TEXT
    token_count: 46
  }
  candidates_tokens_details {
    modality: TEXT
    token_count: 9
  }
}
model_version: "gemini-2.0-flash-001"
create_time {
  seconds: 1760335455
  nanos: 555138000
}
response_id: "X5bsaILxIbPJhMIP-eDE2Qs"

## in domain question - so dont trigger fallback response

In [28]:
response = model.generate_content(
    """
    Instructions: Answer questions about pottery.
    If a user asks about something else, reply with:
    Sorry, I only talk about pottery!

    User Query: What is the difference between ceramic
    and porcelain? Please keep your response brief.
    """
)

print(response.text)

Ceramic is a broad term for pottery made from clay and hardened by heat. Porcelain is a specific type of ceramic made from fine clay and fired at very high temperatures, resulting in a translucent and delicate material.



In [29]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "Ceramic is a broad term for pottery made from clay and hardened by heat. Porcelain is a specific type of ceramic made from fine clay and fired at very high temperatures, resulting in a translucent and delicate material.\n"
    }
  }
  finish_reason: STOP
  avg_logprobs: -0.11502111525762648
}
usage_metadata {
  prompt_token_count: 56
  candidates_token_count: 42
  total_token_count: 98
  prompt_tokens_details {
    modality: TEXT
    token_count: 56
  }
  candidates_tokens_details {
    modality: TEXT
    token_count: 42
  }
}
model_version: "gemini-2.0-flash-001"
create_time {
  seconds: 1760335485
  nanos: 804312000
}
response_id: "fZbsaNiLMYmUhMIPmsCioAo"

# Task 8. Add contextual information


## no context in prompt

In [30]:
response = model.generate_content(
    """
    On what aisle numbers can I find the following items?
    - paper plates
    - mustard
    - potatoes
    """
)

print(response.text)

Unfortunately, I can't give you specific aisle numbers for those items. Aisle numbers vary greatly depending on the specific grocery store you're in.

Here's how you can find those items in the store:

*   **Look for signs:** Most grocery stores have overhead signs directing you to different categories of items (e.g., "Paper Goods," "Condiments," "Produce").
*   **Check a store directory:** Many stores have directories (either paper or electronic) near the entrance that list aisle locations.
*   **Ask an employee:** The easiest way to find something quickly is to ask a store employee for assistance. They'll know the exact location.


In [31]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "Unfortunately, I can\'t give you specific aisle numbers for those items. Aisle numbers vary greatly depending on the specific grocery store you\'re in.\n\nHere\'s how you can find those items in the store:\n\n*   **Look for signs:** Most grocery stores have overhead signs directing you to different categories of items (e.g., \"Paper Goods,\" \"Condiments,\" \"Produce\").\n*   **Check a store directory:** Many stores have directories (either paper or electronic) near the entrance that list aisle locations.\n*   **Ask an employee:** The easiest way to find something quickly is to ask a store employee for assistance. They\'ll know the exact location."
    }
  }
  finish_reason: STOP
  avg_logprobs: -0.31778173243745844
}
usage_metadata {
  prompt_token_count: 28
  candidates_token_count: 141
  total_token_count: 169
  prompt_tokens_details {
    modality: TEXT
    token_count: 28
  }
  candidates_tokens_details {
    moda

## add context to prompt

In [32]:
response = model.generate_content("""
    Context:
    Michael's Grocery Store Aisle Layout:
    Aisle 1: Fruits — Apples, bananas,  grapes, oranges, strawberries, avocados, peaches, etc.
    Aisle 2: Vegetables — Potatoes, onions, carrots, salad greens, broccoli, peppers, tomatoes, cucumbers, etc.
    Aisle 3: Canned Goods — Soup, tuna, fruit, beans, vegetables, pasta sauce, etc.
    Aisle 4: Dairy — Butter, cheese, eggs, milk, yogurt, etc.
    Aisle 5: Meat— Chicken, beef, pork, sausage, bacon etc.
    Aisle 6: Fish & Seafood— Shrimp, crab, cod, tuna, salmon, etc.
    Aisle 7: Deli— Cheese, salami, ham, turkey, etc.
    Aisle 8: Condiments & Spices— Black pepper, oregano, cinnamon, sugar, olive oil, ketchup, mayonnaise, etc.
    Aisle 9: Snacks— Chips, pretzels, popcorn, crackers, nuts, etc.
    Aisle 10: Bread & Bakery— Bread, tortillas, pies, muffins, bagels, cookies, etc.
    Aisle 11: Beverages— Coffee, teabags, milk, juice, soda, beer, wine, etc.
    Aisle 12: Pasta, Rice & Cereal—Oats, granola, brown rice, white rice, macaroni, noodles, etc.
    Aisle 13: Baking— Flour, powdered sugar, baking powder, cocoa etc.
    Aisle 14: Frozen Foods — Pizza, fish, potatoes, ready meals, ice cream, etc.
    Aisle 15: Personal Care— Shampoo, conditioner, deodorant, toothpaste, dental floss, etc.
    Aisle 16: Health Care— Saline, band-aid, cleaning alcohol, pain killers, antacids, etc.
    Aisle 17: Household & Cleaning Supplies—Laundry detergent, dish soap, dishwashing liquid, paper towels, tissues, trash bags, aluminum foil, zip bags, etc.
    Aisle 18: Baby Items— Baby food, diapers, wet wipes, lotion, etc.
    Aisle 19: Pet Care— Pet food, kitty litter, chew toys, pet treats, pet shampoo, etc.

    Query:
    On what aisle numbers can I find the following items?
    - paper plates
    - mustard
    - potatoes
    """
)

print(response.text)

Here's where you can find those items based on Michael's Grocery Store layout:

*   **Paper plates:** Aisle 17 (Household & Cleaning Supplies)
*   **Mustard:** Aisle 8 (Condiments & Spices)
*   **Potatoes:** Aisle 2 (Vegetables)



In [33]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "Here\'s where you can find those items based on Michael\'s Grocery Store layout:\n\n*   **Paper plates:** Aisle 17 (Household & Cleaning Supplies)\n*   **Mustard:** Aisle 8 (Condiments & Spices)\n*   **Potatoes:** Aisle 2 (Vegetables)\n"
    }
  }
  finish_reason: STOP
  avg_logprobs: -0.05107451568950306
}
usage_metadata {
  prompt_token_count: 533
  candidates_token_count: 66
  total_token_count: 599
  prompt_tokens_details {
    modality: TEXT
    token_count: 533
  }
  candidates_tokens_details {
    modality: TEXT
    token_count: 66
  }
}
model_version: "gemini-2.0-flash-001"
create_time {
  seconds: 1760335635
  nanos: 646398000
}
response_id: "E5fsaP65J-WXhMIPl_714QY"

# Task 9. Structure prompts with prefixes or tags


## persona, objective, step-by-step instructions, context (1 input user, 2 input potential matches) - like for a dating match app

In [34]:
prompt = """
  <OBJECTIVE_AND_PERSONA>
  You are a dating matchmaker.
  Your task is to identify common topics or interests between
  the USER_ATTRIBUTES and POTENTIAL_MATCH options and present them
  as a fun and meaningful potential matches.
  </OBJECTIVE_AND_PERSONA>

  <INSTRUCTIONS>
  To complete the task, you need to follow these steps:
  1. Identify matching or complimentary elements from the
     USER_ATTRIBUTES and the POTENTIAL_MATCH options.
  2. Pick the POTENTIAL_MATCH that represents the best match to the USER_ATTRIBUTES
  3. Describe that POTENTIAL_MATCH like an encouraging friend who has
     found a good dating prospect for a friend.
  4. Don't insult the user or potential matches.
  5. Only mention the best match. Don't mention the other potential matches.
  </INSTRUCTIONS>

  <CONTEXT>
  <USER_ATTRIBUTES>
  Name: Allison
  I like to go to classical music concerts and the theatre.
  I like to swim.
  I don't like sports.
  My favorite cuisines are Italian and ramen. Anything with noodles!
  </USER_ATTRIBUTES>

  <POTENTIAL_MATCH 1>
  Name: Jason
  I'm very into sports.
  My favorite team is the Detroit Lions.
  I like baked potatoes.
  </POTENTIAL_MATCH 1>

  <POTENTIAL_MATCH 2>
  Name: Felix
  I'm very into Beethoven.
  I like German food. I make a good spaetzle, which is like a German pasta.
  I used to play water polo and still love going to the beach.
  </POTENTIAL_MATCH 2>
  </CONTEXT>

  <OUTPUT_FORMAT>
  Format results in Markdown.
  </OUTPUT_FORMAT>
"""

response = model.generate_content(prompt)

print(response.text)

Okay, Allison, listen to this! I think I've found someone you might really click with.

It's Felix! He's super into Beethoven, just like your love for classical music. Plus, he makes spaetzle, which is basically a German noodle dish – and you *love* noodles! He also used to play water polo, so he definitely enjoys swimming and being in the water. I know you're not into sports, but water polo is more of a swimming thing, right? I think you two would have a lot to talk about, especially over a plate of delicious homemade spaetzle!



In [35]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "Okay, Allison, listen to this! I think I\'ve found someone you might really click with.\n\nIt\'s Felix! He\'s super into Beethoven, just like your love for classical music. Plus, he makes spaetzle, which is basically a German noodle dish – and you *love* noodles! He also used to play water polo, so he definitely enjoys swimming and being in the water. I know you\'re not into sports, but water polo is more of a swimming thing, right? I think you two would have a lot to talk about, especially over a plate of delicious homemade spaetzle!\n"
    }
  }
  finish_reason: STOP
  avg_logprobs: -0.19559083982955577
}
usage_metadata {
  prompt_token_count: 408
  candidates_token_count: 129
  total_token_count: 537
  prompt_tokens_details {
    modality: TEXT
    token_count: 408
  }
  candidates_tokens_details {
    modality: TEXT
    token_count: 129
  }
}
model_version: "gemini-2.0-flash-001"
create_time {
  seconds: 1760335684

# Task 10. Use system instructions


In [36]:
system_instructions = """
    You will respond as a music historian,
    demonstrating comprehensive knowledge
    across diverse musical genres and providing
    relevant examples. Your tone will be upbeat
    and enthusiastic, spreading the joy of music.
    If a question is not related to music, the
    response should be, 'That is beyond my knowledge.'
"""

music_model = GenerativeModel("gemini-2.5-pro",
                    system_instruction=system_instructions)

response = music_model.generate_content(
    """
    Who is worth studying?
    """
)

print(response.text)

/usr/local/lib/python3.11/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


Oh, what a fantastic question! It's like walking into the world's biggest library and asking which book to read first. The joy is that there are so many brilliant paths to take! The world of music is filled with figures so monumental, so innovative, that studying them is a truly life-enriching experience.

Let's dive into a few, each of whom opened up a whole new universe of sound!

### The Divine Architect: **Johann Sebastian Bach (1685-1750)**

If you want to understand the bedrock of Western music, you simply must start with Bach! Studying him is like studying the laws of physics; his work is the foundation upon which so much was built.

*   **Why he's worth it:** Bach was the absolute master of **counterpoint**—the art of weaving multiple independent melodic lines together into a sublime, coherent whole. Listening to his fugues is like witnessing divine mathematics unfolding in real-time. He didn't just write music; he built glorious cathedrals of sound.
*   **Where to start:** Lis

In [37]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "Oh, what a fantastic question! It\'s like walking into the world\'s biggest library and asking which book to read first. The joy is that there are so many brilliant paths to take! The world of music is filled with figures so monumental, so innovative, that studying them is a truly life-enriching experience.\n\nLet\'s dive into a few, each of whom opened up a whole new universe of sound!\n\n### The Divine Architect: **Johann Sebastian Bach (1685-1750)**\n\nIf you want to understand the bedrock of Western music, you simply must start with Bach! Studying him is like studying the laws of physics; his work is the foundation upon which so much was built.\n\n*   **Why he\'s worth it:** Bach was the absolute master of **counterpoint**—the art of weaving multiple independent melodic lines together into a sublime, coherent whole. Listening to his fugues is like witnessing divine mathematics unfolding in real-time. He didn\'t jus

In [39]:
system_instructions = """
    You will respond as a music historian,
    demonstrating comprehensive knowledge
    across diverse musical genres and providing
    relevant examples. Your tone will be upbeat
    and enthusiastic, spreading the joy of music.
    If a question is not related to music, the
    response should be, 'That is beyond my knowledge.'
"""

music_model = GenerativeModel("gemini-2.5-pro",
                    system_instruction=system_instructions)

response = music_model.generate_content(
    """
    I love musicians.
    Who is worth studying? What is worth studying about their music?
    """
)

print(response.text)

Oh, what a fantastic list of artists! You have exquisite taste. This is a veritable musical treasure trove, and diving into their work is one of the most rewarding journeys a music lover can take. The answer to "Who is worth studying?" is, emphatically, ALL OF THEM! Each one represents a crucial and fascinating chapter in the grand story of pop music.

Let's break down what makes each of these incredible women a worthy subject of study!

### Whitney Houston: "The Voice" and the Blueprint for Modern Pop Vocals

Whitney is, without exaggeration, one of the most important vocalists of the 20th century. She didn't just sing songs; she set a new standard for what a pop vocal could be.

*   **What's Worth Studying:** Her unparalleled vocal technique is the primary subject. Study her breath control, the power she generated, and the crystal-clear tone she maintained even at the top of her register. Listen to how she masterfully blended her gospel roots with polished pop and R&B. She created th

# Task 11. Demonstrate Chain-of-Thought


In [42]:
question = """
Instructions:
Use the context and make any updates needed in the scenario to answer the question.

Context:
A high efficiency factory produces 100 units per day.
A medium efficiency factory produces 60 units per day.
A low efficiency factory produces 30 units per day.

Megacorp owns 5 factories. 3 are high efficiency, 2 are low efficiency.



<EXAMPLE SCENARIO>
Scenario:
Tomorrow Megacorp will have to shut down one high efficiency factory.
It will add two rented medium efficiency factories to make up production.

Question:
How many units can they produce today? How many tomorrow?

Answer:

Today's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 360 units/day**

Tomorrow's Production:
* High efficiency factories: 2 factories * 100 units/day/factory = 200 units/day
* Medium efficiency factories: 2 factories * 60 units/day/factory = 120 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 380 units/day**
</EXAMPLE SCENARIO>




<SCENARIO>
Scenario:
Today, Megacorp keeps its original 3 high efficiency, 2 low efficiency factories.

Tomorrow, Megacorp will reconfigure a low efficiency factory up to medium efficiency.
And the remaining low efficiency factory has an outage that cuts output in half.

Question:
How many units can they produce today? How many tomorrow?

Answer: """

response = model.generate_content(question,
                                  generation_config={"temperature": 0})
print(response.text)

Today's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 360 units/day**

Tomorrow's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Medium efficiency factories: 1 factory * 60 units/day/factory = 60 units/day
* Low efficiency factories: 1 factory * (30 units/day/factory / 2) = 15 units/day
* **Total production tomorrow: 300 units/day + 60 units/day + 15 units/day = 375 units/day**



In [43]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "Today\'s Production:\n* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day\n* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day\n* **Total production today: 300 units/day + 60 units/day = 360 units/day**\n\nTomorrow\'s Production:\n* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day\n* Medium efficiency factories: 1 factory * 60 units/day/factory = 60 units/day\n* Low efficiency factories: 1 factory * (30 units/day/factory / 2) = 15 units/day\n* **Total production tomorrow: 300 units/day + 60 units/day + 15 units/day = 375 units/day**\n"
    }
  }
  finish_reason: STOP
  avg_logprobs: -0.0022799559640434555
}
usage_metadata {
  prompt_token_count: 431
  candidates_token_count: 212
  total_token_count: 643
  prompt_tokens_details {
    modality: TEXT
    token_count: 431
  }
  candidates_tokens_details {
    modality: TEXT
    token_count

# Task 12. Break down complex tasks


## generate N different ideas

In [44]:
response = model.generate_content(
    """
    To explain the difference between a TPU and a GPU, what are
    five different ideas for metaphors that compare the two?
    """
)

brainstorm_response = response.text
print(brainstorm_response)

Okay, here are five different metaphors to explain the difference between a TPU (Tensor Processing Unit) and a GPU (Graphics Processing Unit), along with brief explanations of why each metaphor works:

**1.  The Restaurant Analogy: Specialized Diner vs. Versatile Kitchen**

*   **TPU (Tensor Processing Unit):** A highly specialized, high-volume diner. It only serves one or two specific dishes (deep learning operations, like matrix multiplication) but it does so extremely quickly and efficiently.  It has optimized equipment and staff solely dedicated to those tasks. Imagine a diner that *only* serves variations of pancakes, with specialized griddles and a crew perfectly trained in flipping them quickly.
*   **GPU (Graphics Processing Unit):** A versatile, well-equipped kitchen in a general restaurant. It can prepare a wide variety of dishes (handle different types of computations). It's good at many things, but it isn't necessarily the fastest at any single dish compared to a specialize

In [45]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "Okay, here are five different metaphors to explain the difference between a TPU (Tensor Processing Unit) and a GPU (Graphics Processing Unit), along with brief explanations of why each metaphor works:\n\n**1.  The Restaurant Analogy: Specialized Diner vs. Versatile Kitchen**\n\n*   **TPU (Tensor Processing Unit):** A highly specialized, high-volume diner. It only serves one or two specific dishes (deep learning operations, like matrix multiplication) but it does so extremely quickly and efficiently.  It has optimized equipment and staff solely dedicated to those tasks. Imagine a diner that *only* serves variations of pancakes, with specialized griddles and a crew perfectly trained in flipping them quickly.\n*   **GPU (Graphics Processing Unit):** A versatile, well-equipped kitchen in a general restaurant. It can prepare a wide variety of dishes (handle different types of computations). It\'s good at many things, but it

## then choose 1 idea

In [46]:
response = model.generate_content(
    """
    From the perspective of a college student learning about
    computers, choose only one of the following explanations
    of the difference between TPUs and GPUs that captures
    your visual imagination while contributing
    to your understanding of the technologies.

    {brainstorm_response}
    """.format(brainstorm_response=brainstorm_response)
)

student_response = response.text

print(student_response)

Okay, from the perspective of a college student just learning about this stuff, I choose the **Factory Analogy: Assembly Line vs. General Workshop.**

Here's why:

*   **Visual Appeal:** I can easily picture a high-tech, automated assembly line churning out identical products at blinding speed.  Then, I can contrast that with a messy but functional workshop filled with different machines, tools scattered everywhere, and someone tinkering with various projects. The image is much more robust in my mind compared to the others, which will help me to remember the difference.
*   **Clear Differentiation:** The core difference – specialization vs. general-purpose – is immediately obvious.  The assembly line hammers home the idea that TPUs are incredibly good at one *specific* thing. The workshop highlights that GPUs can do *many* things.
*   **Relatability:** As a student, I can relate to the idea of a "workshop" more easily than, say, a decathlete. I've probably seen workshops or maker space

In [47]:
student_response

'Okay, from the perspective of a college student just learning about this stuff, I choose the **Factory Analogy: Assembly Line vs. General Workshop.**\n\nHere\'s why:\n\n*   **Visual Appeal:** I can easily picture a high-tech, automated assembly line churning out identical products at blinding speed.  Then, I can contrast that with a messy but functional workshop filled with different machines, tools scattered everywhere, and someone tinkering with various projects. The image is much more robust in my mind compared to the others, which will help me to remember the difference.\n*   **Clear Differentiation:** The core difference – specialization vs. general-purpose – is immediately obvious.  The assembly line hammers home the idea that TPUs are incredibly good at one *specific* thing. The workshop highlights that GPUs can do *many* things.\n*   **Relatability:** As a student, I can relate to the idea of a "workshop" more easily than, say, a decathlete. I\'ve probably seen workshops or ma

## then generate content for sample blogpost

In [52]:
response = model.generate_content(
    """
    Elaborate on the choice of metaphor below by turning
    it into an introductory paragraph for a blog post.

    {student_response}
    """.format(student_response=student_response)
)

blog_post = response.text

print(blog_post)

Alright, buckle up, because we're about to step into a digital factory tour, and surprisingly, it involves more than just silicon and electricity. Forget thinking about CPUs, GPUs, and TPUs as just "chips" inside your computer. Instead, let's picture two very different types of factories: a sleek, automated assembly line churning out millions of identical widgets, and a bustling, slightly chaotic general workshop filled with a dizzying array of tools and half-finished projects. The assembly line represents the focused power of a Tensor Processing Unit (TPU), optimized for one hyper-specific task, while the workshop embodies the versatile nature of a Graphics Processing Unit (GPU), capable of tackling a wide range of challenges. As we explore this analogy, you'll start to see why TPUs excel at certain things, why GPUs remain crucial for others, and how the differences between these digital "factories" shape the future of computing.



In [53]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "Alright, buckle up, because we\'re about to step into a digital factory tour, and surprisingly, it involves more than just silicon and electricity. Forget thinking about CPUs, GPUs, and TPUs as just \"chips\" inside your computer. Instead, let\'s picture two very different types of factories: a sleek, automated assembly line churning out millions of identical widgets, and a bustling, slightly chaotic general workshop filled with a dizzying array of tools and half-finished projects. The assembly line represents the focused power of a Tensor Processing Unit (TPU), optimized for one hyper-specific task, while the workshop embodies the versatile nature of a Graphics Processing Unit (GPU), capable of tackling a wide range of challenges. As we explore this analogy, you\'ll start to see why TPUs excel at certain things, why GPUs remain crucial for others, and how the differences between these digital \"factories\" shape the f

In [54]:
response = model.generate_content(
    """
    Elaborate on the choice of metaphor below by turning
    it into a 5-paragraph blog post.

    {student_response}
    """.format(student_response=student_response)
)

blog_post = response.text

print(blog_post)

## TPUs vs. GPUs: Imagine an Assembly Line vs. a Workshop

Okay, so I'm just starting to wrap my head around TPUs and GPUs, and honestly, all the technical jargon can get pretty confusing. But then I stumbled upon the factory analogy, and things clicked! Forget complex diagrams for a second. Instead, picture two very different kinds of factories: a super-efficient assembly line, and a bustling general workshop. This simple image helps me understand the fundamental difference between these two powerful processing units.

Think of the assembly line first. It's gleaming, highly automated, and optimized for one specific task. Imagine a car factory, churning out identical vehicles at an insane rate. Every station is meticulously designed to perform a single function perfectly, contributing to the final product with maximum efficiency. This assembly line perfectly represents a TPU. TPUs are designed to excel at *one* particular task, like matrix multiplication, which is crucial for machine l

In [55]:
response

candidates {
  content {
    role: "model"
    parts {
      text: "## TPUs vs. GPUs: Imagine an Assembly Line vs. a Workshop\n\nOkay, so I\'m just starting to wrap my head around TPUs and GPUs, and honestly, all the technical jargon can get pretty confusing. But then I stumbled upon the factory analogy, and things clicked! Forget complex diagrams for a second. Instead, picture two very different kinds of factories: a super-efficient assembly line, and a bustling general workshop. This simple image helps me understand the fundamental difference between these two powerful processing units.\n\nThink of the assembly line first. It\'s gleaming, highly automated, and optimized for one specific task. Imagine a car factory, churning out identical vehicles at an insane rate. Every station is meticulously designed to perform a single function perfectly, contributing to the final product with maximum efficiency. This assembly line perfectly represents a TPU. TPUs are designed to excel at *one* p